IMPORT DATA

In [55]:
##grab all of the Kaggle Data

Kaggle_files = ['game.csv',
'game_teams_stats.csv',
'game_scratches.csv',
'game_officials.csv',
'game_skater_stats.csv',
'game_goalie_stats.csv'] 


Kaggle_path = "/Users/joejohns/data_bootcamp/Final_Project_NHL_prediction/Data/Kaggle_Data_Ellis/"
mp_path = "/Users/joejohns/data_bootcamp/Final_Project_NHL_prediction/Data/Money_Puck_Data/"
betting_path = "/Users/joejohns/data_bootcamp/Final_Project_NHL_prediction/Data/Betting_Data/"


import pandas as pd
import numpy as np

In [56]:
df_game = pd.read_csv(Kaggle_path+'game.csv')
df_game_team_stats = pd.read_csv(Kaggle_path+'game_teams_stats.csv')
df_game_skater_stats = pd.read_csv(Kaggle_path+'game_skater_stats.csv')
df_game_goalie_stats = pd.read_csv(Kaggle_path+'game_goalie_stats.csv')

#team_id---> team_names (there is one for player_ids too)
df_team_info = pd.read_csv(Kaggle_path+'team_info.csv')


##more subtle features:
df_game_scratches = pd.read_csv(Kaggle_path+'game_scratches.csv')
df_game_officials = pd.read_csv(Kaggle_path+'game_officials.csv')




In [57]:
## grab all the moneypuck data 

df_mp_teams = pd.read_csv(mp_path+'all_teams.csv')




In [58]:
## grab all betting data 
df1 = pd.read_excel(io = betting_path+'nhl odds 2007-08.xlsx')
df2 = pd.read_excel(io = betting_path+'nhl odds 2008-09.xlsx')
df3 = pd.read_excel(io = betting_path+'nhl odds 2009-10.xlsx')
df4 = pd.read_excel(io = betting_path+'nhl odds 2010-11.xlsx')
df5 = pd.read_excel(io = betting_path+'nhl odds 2011-12.xlsx')
df6 = pd.read_excel(io = betting_path+'nhl odds 2012-13.xlsx')
df7 = pd.read_excel(io = betting_path+'nhl odds 2013-14.xlsx')
df8 = pd.read_excel(io = betting_path+'nhl odds 2014-15.xlsx')
df9 = pd.read_excel(io = betting_path+'nhl odds 2015-16.xlsx')
df10 = pd.read_excel(io = betting_path+'nhl odds 2016-17.xlsx')
df11 = pd.read_excel(io = betting_path+'nhl odds 2017-18.xlsx')
df12 = pd.read_excel(io = betting_path+'nhl odds 2018-19.xlsx')
df13 = pd.read_excel(io = betting_path+'nhl odds 2019-20.xlsx')




In [59]:
df1['season'] = 20072008
df2['season'] = 20082009
df3['season'] = 20092010
df4['season'] = 20102011
df5['season'] = 20112012
df6['season'] = 20122013
df7['season'] = 20132014
df8['season'] = 20142015
df9['season'] = 20152016
df10['season'] = 20162017
df11['season'] = 20172018
df12['season'] = 20182019
df13['season'] = 20192020




In [60]:
##omit 2007-2008

df_betting = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13])

In [61]:
##drop diplicates here 

df_game.drop_duplicates(inplace = True)

In [62]:
## restrict data sets 
df_betting = df_betting.loc[:, ['Date', 'season','VH', 'Team', 'Open']].copy()



df_mp_teams_all = df_mp_teams.loc[df_mp_teams['situation'] == 'all', :].copy()

FIX SEASONS IN df_mp ... then all 3 of df_games, df_mp_teams, df_betting have same season format for cutting down seasons

In [63]:
#df_mp_teams_all['']
def fix_mp_season(n):
    return int(str(n)+str(n+1))

#test fix_mp_season(2008)
df_mp_teams['season'] = df_mp_teams['season'].map(fix_mp_season)

In [64]:

##restrict seasons 
seasons = []
for n in range(2008,2020):
    seasons.append(int(str(n)+str(n+1)))

print(seasons)


[20082009, 20092010, 20102011, 20112012, 20122013, 20132014, 20142015, 20152016, 20162017, 20172018, 20182019, 20192020]


In [65]:
for seas in seasons:
    print(seas, len(df_mp_teams.loc[df_mp_teams['season']==seas])/10, len(df_game.loc[df_game['season']==seas]), len(df_betting.loc[df_betting['season']==seas])/2)

20082009 1314.0 1230 1317.0
20092010 1318.0 1230 1318.0
20102011 1318.0 1319 1319.0
20112012 1316.0 1316 1315.0
20122013 806.0 806 806.0
20132014 1323.0 1323 1323.0
20142015 1319.0 1319 1319.0
20152016 1321.0 1321 1321.0
20162017 1317.0 1323 1317.0
20172018 1355.0 1363 1355.0
20182019 1358.0 1360 1358.0
20192020 1212.0 1215 1212.0


In [ ]:
##they match up starting 2013-14 ... but that was short so let's go with 2013-14

CUT DOWN SEASONS TO 2008-09 to 2019-20

In [66]:
#restrict seasons: #betting is done already

df_game = df_game.loc[df_game['season'].isin(seasons), :].copy()
df_mp_teams = df_mp_teams.loc[df_mp_teams['season'].isin(seasons), :].copy()

FIX TEAM NAMES

In [67]:
##fix names of teams
fix_dic = {'Arizonas': 'Arizona', 'NYIslanders': 'NY Islanders', 'NYRangers': 'NY Rangers',
           'NewJersey':'New Jersey', 'LosAngeles': 'Los Angeles', 'SanJose': 'San Jose', 'St.Louis': 'St Louis', 
           'Tampa': 'Tampa Bay',  'TampaBay': 'Tampa Bay', 'WinnipegJets': 'Winnipeg'}



def fix_bet_names(s):
    if s in fix_dic.keys():
        return fix_dic[s]
    else:
        return s

def fix_mp_names(s):
    if s == 'L.A':
        return 'LAK'
    elif s == 'S.J':
        return 'SJS'
    elif s == 'N.J':
        return 'NJD'
    elif s == 'T.B':
        return 'TBL'
    else:
        return s

In [68]:
df_betting['Team'] = df_betting['Team'].apply(fix_bet_names)

df_mp_teams['playerTeam']  = df_mp_teams['playerTeam'].apply(fix_mp_names)  
df_mp_teams['opposingTeam']  = df_mp_teams['opposingTeam'].apply(fix_mp_names) 

ATTACH team_id to betting


In [69]:

def attach_bet_team_id(s):
    return df_team_info.loc[df_team_info['shortName'] == s, 'team_id'].array[0]

In [70]:
df_betting['team_id'] = df_betting['Team'].map(attach_bet_team_id) 

put proper dates on df_game and df_betting


Attach game_id to df_betting; 
To do this we need to attach a smilar month day code to df_game so we can match them


In [95]:
'20082009'[4:]

'2009'

In [98]:
#restrict seasons: #betting is done already
def bet_mp_date(date, season):
    d = str(date)
    s = str(season)
    if len(d) == 3:
        d = "0"+d 
    if (900 < date):
        y= s[:4]
    else:
        y = s[4:]
    return int(y+d)


In [99]:
df_betting['mp_date'] = np.vectorize(bet_mp_date)(df_betting['Date'], df_betting['season'])

In [110]:
df_betting.iloc[2000:2050, :]

,Date,season,VH,Team,Open,team_id,mp_date
2000,304,20072008,V,Pittsburgh,-115,5,20080304
2001,304,20072008,H,Tampa Bay,-105,14,20080304
2002,304,20072008,V,New Jersey,-130,1,20080304
2003,304,20072008,H,Toronto,110,10,20080304
2004,304,20072008,V,Chicago,171,16,20080304
2005,304,20072008,H,Minnesota,-201,30,20080304
2006,304,20072008,V,Los Angeles,130,26,20080304
2007,304,20072008,H,St Louis,-150,19,20080304
2008,304,20072008,V,Columbus,155,29,20080304
2009,304,20072008,H,Calgary,-175,20,20080304


In [71]:
## do mp style date for df_games

df_game['date_time_GMT'] = pd.to_datetime(df_game['date_time_GMT']) 

In [103]:
def to_bet_date(date_time):
    d = str(date_time.day)
    m = str(date_time.month)
    #y = str(date_time.year)
    if len(d) ==1:
        d = "0"+d
    if len(m) ==1:
        m = "0"+m
    return int(m+d)

In [101]:
def to_mp_date(date_time):
    d = str(date_time.day)
    m = str(date_time.month)
    y = str(date_time.year)
    if len(d) ==1:
        d = "0"+d
    if len(m) ==1:
        m = "0"+m
    return int(y+m+d)

        

In [104]:
test = df_game['date_time_GMT'][0]
print(test, to_bet_date(test), to_mp_date(test))


2016-10-19 00:30:00+00:00 1019 20161019


In [24]:
#test to_bet_date(18, 9)  to_bet_date(8, 10)

In [106]:
df_game['mp_date'] = df_game['date_time_GMT'].apply(to_mp_date)

In [107]:
df_mp_teams.iloc[700: 750, :]

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
700,NYR,20092010,NYR,2009020861,NYR,NJD,HOME,20100206,Team Level,other,...,7.0,7.000,0.0,0.139,0.000,0.000,1.122,1.122,0.779,0
701,NYR,20092010,NYR,2009020861,NYR,NJD,HOME,20100206,Team Level,all,...,54.0,52.843,3.0,0.649,1.491,1.491,3.808,3.783,3.290,0
702,NYR,20092010,NYR,2009020861,NYR,NJD,HOME,20100206,Team Level,5on5,...,37.0,35.843,2.0,0.390,1.161,1.161,2.338,2.313,2.166,0
703,NYR,20092010,NYR,2009020861,NYR,NJD,HOME,20100206,Team Level,4on5,...,4.0,4.000,1.0,0.060,0.000,0.000,0.084,0.084,0.084,0
704,NYR,20092010,NYR,2009020861,NYR,NJD,HOME,20100206,Team Level,5on4,...,6.0,6.000,0.0,0.059,0.330,0.330,0.264,0.264,0.261,0
705,NYR,20092010,NYR,2009020884,NYR,NSH,HOME,20100210,Team Level,other,...,0.0,0.000,2.0,0.000,0.000,0.000,0.000,0.000,0.000,0
706,NYR,20092010,NYR,2009020884,NYR,NSH,HOME,20100210,Team Level,all,...,46.0,48.675,4.0,0.477,0.033,0.055,3.879,4.039,3.763,0
707,NYR,20092010,NYR,2009020884,NYR,NSH,HOME,20100210,Team Level,5on5,...,35.0,37.675,1.0,0.297,0.016,0.039,1.848,2.008,1.985,0
708,NYR,20092010,NYR,2009020884,NYR,NSH,HOME,20100210,Team Level,4on5,...,10.0,10.000,1.0,0.174,0.016,0.016,1.984,1.984,1.731,0
709,NYR,20092010,NYR,2009020884,NYR,NSH,HOME,20100210,Team Level,5on4,...,1.0,1.000,0.0,0.006,0.000,0.000,0.047,0.047,0.047,0


In [ ]:
def to_mp_date(date_time):
    d = str(date_time.day)
    m = str(date_time.month)
    if len(d) ==1:
        d = "0"+d
    return int(m+d)

In [76]:
def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type', 'Percent'])
    return missing_data

In [74]:
df_game.drop(columns = ['home_rink_side_start'], inplace = True)

In [77]:
perc_null(df_game)

,Total,Type,Percent
game_id,0,int64,0.0
season,0,int64,0.0
type,0,object,0.0
date_time_GMT,0,"datetime64[ns, UTC]",0.0
away_team_id,0,int64,0.0
home_team_id,0,int64,0.0
away_goals,0,int64,0.0
home_goals,0,int64,0.0
outcome,0,object,0.0
venue,0,object,0.0


In [27]:
#test

len(df_game.loc[(df_game['bet_date'] == 1004)&(df_game['season'] == 20082009)&(df_game['home_team_id'] == 4),  :]['game_id'].values)

0

In [28]:
df_betting.loc[df_betting['VH']=='N',:]

,Date,season,VH,Team,Open,team_id
2164,801,20192020,N,Montreal,-110,8
2165,801,20192020,N,Pittsburgh,-120,5
2166,801,20192020,N,NY Rangers,120,3
2167,801,20192020,N,Carolina,-150,12
2168,801,20192020,N,Florida,110,13
...,...,...,...,...,...,...
2419,925,20192020,N,Dallas,140,25
2420,926,20192020,N,Dallas,145,25
2421,926,20192020,N,Tampa Bay,-166,14
2422,928,20192020,N,Tampa Bay,-161,14


In [123]:
def assign_game_id(mp_date, team_id): #this is assuming it is home team and these are from df_betting 
    h = df_game.loc[(df_game['mp_date'] == mp_date)&(df_game['home_team_id'] == team_id),  :]['game_id'].values
    a = df_game.loc[(df_game['mp_date'] == mp_date)&(df_game['away_team_id'] == team_id),  :]['game_id'].values
    if len(h) == 1:
        return h[0]
    elif len(a) == 1:
        return a[0]
    elif len(a) > 1:
        return 2
    elif len(a) < 1: 
        return 0
    elif len(h) > 1:
        return 3
    elif len(h)< 1:
        return 1
       

In [30]:
df_game.loc[(df_game['bet_date'] == 1009)&(df_game['season'] == 20082009)&(df_game['away_team_id'] == 6),  :]['game_id'].values


#1009	20082009	V	Boston	120	6	314159

array([], dtype=int64)

In [120]:
df_mp_teams.loc[(df_mp_teams['gameDate']==20081010)&(df_mp_teams['situation']=='all'), :]

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
11,NYR,20082009,NYR,2008020010,NYR,CHI,HOME,20081010,Team Level,all,...,45.0,43.911,3.0,0.448,0.407,0.407,2.738,2.751,2.730,0
25586,DAL,20082009,DAL,2008020015,DAL,CBJ,HOME,20081010,Team Level,all,...,34.0,35.088,10.0,0.419,0.429,0.429,2.273,2.400,2.369,0
30846,N.J,20082009,N.J,2008020009,NJD,NYI,HOME,20081010,Team Level,all,...,33.0,32.719,5.0,0.412,0.244,0.283,2.481,2.497,2.389,0
64151,NYI,20082009,NYI,2008020009,NYI,NJD,AWAY,20081010,Team Level,all,...,42.0,42.375,0.0,0.449,0.570,0.570,2.125,2.083,2.065,0
69381,MTL,20082009,MTL,2008020012,MTL,BUF,AWAY,20081010,Team Level,all,...,47.0,45.885,9.0,0.530,0.000,0.000,3.042,2.948,2.902,0
74756,STL,20082009,STL,2008020014,STL,NSH,HOME,20081010,Team Level,all,...,39.0,38.176,1.0,0.354,0.131,0.131,1.745,1.766,1.762,0
80216,FLA,20082009,FLA,2008020011,FLA,CAR,AWAY,20081010,Team Level,all,...,39.0,39.178,4.0,0.426,0.631,0.631,2.773,2.723,2.704,0
85266,BUF,20082009,BUF,2008020012,BUF,MTL,HOME,20081010,Team Level,all,...,46.0,47.177,6.0,0.478,0.328,0.328,2.532,2.606,2.594,0
100656,WSH,20082009,WSH,2008020013,WSH,ATL,AWAY,20081010,Team Level,all,...,32.0,32.338,2.0,0.359,0.605,0.605,2.238,2.227,2.210,0
106266,CBJ,20082009,CBJ,2008020015,CBJ,DAL,AWAY,20081010,Team Level,all,...,45.0,44.382,6.0,0.615,0.762,0.762,3.353,3.304,3.117,0


In [119]:
df_betting.loc[(df_betting['Date'] == 1010)& (df_betting['season'] == 20082009), :]

,Date,season,VH,Team,Open,team_id,mp_date,game_id
16,1010,20082009,V,NY Islanders,190,2,20081010,2008020009
17,1010,20082009,H,New Jersey,-230,1,20081010,2008020009
18,1010,20082009,V,Florida,150,13,20081010,2008020011
19,1010,20082009,H,Carolina,-170,12,20081010,2008020011
20,1010,20082009,V,Chicago,125,16,20081010,2008020010
21,1010,20082009,H,NY Rangers,-145,3,20081010,2008020010
22,1010,20082009,V,Montreal,100,8,20081010,2008020012
23,1010,20082009,H,Buffalo,-120,7,20081010,2008020012
24,1010,20082009,V,Washington,-120,15,20081010,2008020013
25,1010,20082009,H,Atlanta,100,11,20081010,2008020013


In [118]:
df_game.loc[(df_game['season'] == 20082009) & (df_game['date_time_GMT'].apply(lambda x : x.month) == 10)&(df_game['date_time_GMT'].apply(lambda x : x.day) == 10), :]

,game_id,season,type,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz,bet_date,mp_date
10630,2008020008,20082009,R,2008-10-10 02:30:00+00:00,24,28,1,4,home win REG,HP Pavilion at San Jose,/api/v1/venues/null,America/Los_Angeles,-8,PST,1010,20081010
11230,2008020013,20082009,R,2008-10-10 23:30:00+00:00,15,11,4,7,home win REG,Philips Arena,/api/v1/venues/null,America/New_York,-5,EST,1010,20081010
11241,2008020011,20082009,R,2008-10-10 23:00:00+00:00,13,12,4,6,home win REG,RBC Center,/api/v1/venues/null,America/New_York,-5,EST,1010,20081010
11247,2008020009,20082009,R,2008-10-10 23:00:00+00:00,2,1,1,2,home win REG,Prudential Center,/api/v1/venues/5067,America/New_York,-5,EST,1010,20081010
11277,2008020010,20082009,R,2008-10-10 23:00:00+00:00,16,3,2,4,home win REG,Madison Square Garden,/api/v1/venues/5054,America/New_York,-5,EST,1010,20081010
11488,2008020012,20082009,R,2008-10-10 23:30:00+00:00,8,7,1,2,home win OT,HSBC Arena,/api/v1/venues/null,America/New_York,-5,EST,1010,20081010
11588,2008020007,20082009,R,2008-10-10 02:30:00+00:00,20,23,0,6,home win REG,General Motors Place,/api/v1/venues/null,America/Vancouver,-8,PST,1010,20081010
11753,2008020006,20082009,R,2008-10-10 02:00:00+00:00,6,21,5,4,away win REG,Pepsi Center,/api/v1/venues/null,America/Denver,-7,MST,1010,20081010


In [ ]:
df_game.loc[(df_game['season'] == 20082009) & (df_game['bet_date'] == 1009), :]

In [31]:
def assign_home_game_id(date, season, team_id): #this is assuming it is home team and these are from df_betting 
    return df_game.loc[(df_game['bet_date'] == date)&(df_game['season'] == season)&(df_game['home_team_id'] == team_id),  :]['game_id'].values[0]

def assign_away_game_id(date, season, team_id):
    return df_game.loc[(df_game['bet_date'] == date)&(df_game['season'] == season)&(df_game['away_team_id'] == team_id),  :]['game_id'].values[0]
    

In [ ]:
assign_away_game_id(1004, 20082009, 3)

In [ ]:
df_betting_home = df_betting.loc[df_betting['VH'] == 'H', :].copy()
df_betting_away = df_betting.loc[df_betting['VH'] == 'V', :].copy()

In [113]:
df_betting['team_id']

0       24
1       26
2       26
3       24
4       24
        ..
2419    25
2420    25
2421    14
2422    14
2423    25
Name: team_id, Length: 33190, dtype: int64

In [124]:
df_betting['game_id'] =  np.vectorize(assign_game_id)(df_betting['mp_date'], df_betting['team_id']) 

In [ ]:
df_betting.shape

In [127]:
(df_betting['game_id'] == 2).sum()

120

In [128]:
(df_betting['game_id'] == 3).sum()

0

In [129]:
(df_betting['game_id'] == 0).sum()

21632

In [130]:
(df_betting['game_id'] == 1).sum()

0

In [1]:
df_game_team_stats.columns

NameError: name 'df_game_team_stats' is not defined